# Handling [tensors](https://pytorch.org/docs/stable/tensors.html?highlight=tensor#torch.Tensor) with PyTorch
At its core, PyTorch is a library that provides multidimensional arrays, called tensors in
PyTorch parlance, and an extensive library of operations on them is provided by the torch module. Both tensors and related operations can run on the CPU or GPU. The second core thing that PyTorch allows tensors to keep track of the operations performed on them and to compute derivatives of an output with respect to any of its inputs analytically via backpropagation.

**Key points**

- Numbers in Python are full-fledged objects.
- Lists in Python are meant for sequential collections of objects.
- The Python interpreter is slow compared with optimized, compiled code

![memory](./images/memory.png)

PyTorch tensors or NumPy arrays, on the other hand, are views over (typically) contiguous memory blocks containing unboxed C numeric types, not Python objects. So a 1D tensor of 1 million float numbers requires 4 million contiguous bytes to be stored, plus a small overhead for the metadata (dimensions, numeric type, and so on).



In [1]:
import numpy as np
import torch
print(torch.__version__) 

1.4.0+cpu


### Pytorch-Numpy interoperability

Values are allocated in contiguous chunks of memory, managed by torch.Storage instances. A storage is a one-dimensional array of numerical data, such as a contiguous block of memory containing numbers of a given type, perhaps a float or int32. A PyTorch Tensor is a view over such a Storage that’s capable of indexing into that storage by using an offset and per-dimension strides

![storage](./images/storage.png)


Some useful methods:
- .tensor() will make a copy of the argument
- .numpy() will return a view on the same memory elements
- .from_numpy() will return a view on the same memory elements

In [2]:
arr_np = np.random.randn(2, 3)
print("Numpy array:\n{}\n".format(arr_np))

tens = torch.tensor(arr_np) # change printing precision with torch.set_printoptions()
print("Tensor :\n{}\n".format(tens))


print("#####\nWe modify the element (0, 0) of the tensor object")
tens[0, 0] = 10
print("Tensor :\n{}\n".format(tens))
print("Numpy array:\n{}\n".format(arr_np))

Numpy array:
[[-1.52770622 -1.20341779  1.11211442]
 [ 0.64405812  1.32827795 -1.96574321]]

Tensor :
tensor([[-1.5277, -1.2034,  1.1121],
        [ 0.6441,  1.3283, -1.9657]], dtype=torch.float64)

#####
We modify the element (0, 0) of the tensor object
Tensor :
tensor([[10.0000, -1.2034,  1.1121],
        [ 0.6441,  1.3283, -1.9657]], dtype=torch.float64)

Numpy array:
[[-1.52770622 -1.20341779  1.11211442]
 [ 0.64405812  1.32827795 -1.96574321]]



In [3]:
arr_from_tens = tens.numpy()
print("Numpy array obtained with tens.numpy():\n{}\n".format(arr_from_tens)) # it is a view on the same storage

arr_from_tens[0, 1] = 20
print("Changing this numpy array will change our tensor too:\n{}\n".format(tens))

Numpy array obtained with tens.numpy():
[[10.         -1.20341779  1.11211442]
 [ 0.64405812  1.32827795 -1.96574321]]

Changing this numpy array will change our tensor too:
tensor([[10.0000, 20.0000,  1.1121],
        [ 0.6441,  1.3283, -1.9657]], dtype=torch.float64)



With slicing we get different tensor views on the same underlying data. We can get copied pieces of those view as python numbers or lists with tens.item() for single values, or tens.tolist().
To check that they are copies, we can use np.shares_memory()

In [4]:
tens_first_row = tens[0]
tens_element_as_py_obj = tens[0, 0].item()
tens_as_list = tens.tolist()

print(np.shares_memory(tens, tens_first_row))
print(np.shares_memory(tens, tens_element_as_py_obj))
print(np.shares_memory(tens, tens_as_list))

True
False
False


The shape of our tensor is given by the way we strides along the underlying storage

In [5]:
print(tens.size())
print(tens.shape)

torch.Size([2, 3])
torch.Size([2, 3])


### [Factory functions](https://pytorch.org/cppdocs/notes/tensor_creation.html#picking-a-factory-function)

In [6]:
print(torch.randint(0, 10, size=(2, 1)))
print(torch.arange(10))
print(torch.linspace(0, 1, 8))
print(torch.eye(2))
print(torch.zeros(3,3))
print(torch.ones(1, 2))
print(torch.randn(3))
print(torch.randperm(12))

tensor([[0],
        [6]])
tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
tensor([0.0000, 0.1429, 0.2857, 0.4286, 0.5714, 0.7143, 0.8571, 1.0000])
tensor([[1., 0.],
        [0., 1.]])
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
tensor([[1., 1.]])
tensor([-0.7216,  0.5356,  0.0401])
tensor([ 6,  7,  3,  9, 10,  5,  0, 11,  1,  8,  2,  4])


In [7]:
print(torch.FloatTensor([[1, 0], [0, 1]]))
print(torch.Tensor(2, 3)) # alias for FloatTensor

tensor([[1., 0.],
        [0., 1.]])
tensor([[-1.1327e-13,  4.5872e-41,  1.7647e-37],
        [ 0.0000e+00,  4.4842e-44,  0.0000e+00]])


torch.empty() allows to construct a tensor without initializing any value. 

We can sample in place from a broader range of distributions by using Tensor's methods.

In [8]:
print(torch.empty(2, 3))
print(torch.empty(3).cauchy_())
print(torch.empty(3).geometric_(0.7))

tensor([[ 0.0000e+00,  0.0000e+00,  5.3255e-38],
        [ 1.9392e-37,  1.7228e-34, -1.1839e+36]])
tensor([-1.2246,  0.8426, -0.9575])
tensor([13.,  4.,  2.])


Trailing underscore concerns in-place methods, i.e. those methods who transforms the input without making a copy

In [9]:
twotwo = 2 * torch.ones(2)
print(twotwo)

twotwo_sqrt = torch.sqrt(twotwo)
print(twotwo_sqrt)

print(np.shares_memory(twotwo, twotwo_sqrt))

twotwo.sqrt_()
print(twotwo)

tensor([2., 2.])
tensor([1.4142, 1.4142])
False
tensor([1.4142, 1.4142])


### [Numeric types](https://pytorch.org/docs/stable/tensors.html#torch-tensor) and casting
The dtype argument to tensor constructors (that is, functions such as tensor, zeros, and ones) specifies the numerical data type that will be contained in the tensor. The data type specifies the possible values that the tensor can hold (integers versus floating-point numbers) and the number of bytes per value.

torch.Tensor is an alias for the default tensor type, torch.FloatTensor

In [10]:
print(torch.zeros(2).type())
print(torch.tensor([0, 1, 2], dtype=torch.short))
print(torch.tensor([0, 1, 2]).short())

print(torch.ones(3, dtype=torch.double))
print(torch.ones(3).double())

torch.FloatTensor
tensor([0, 1, 2], dtype=torch.int16)
tensor([0, 1, 2], dtype=torch.int16)
tensor([1., 1., 1.], dtype=torch.float64)
tensor([1., 1., 1.], dtype=torch.float64)


In addition to the dtype, a PyTorch tensor has a notion of device, which specifies if the operations on that tensor will be carried out by the CPU or a GPU, if one is present on the machine

In [11]:
print(torch.ones(3, device='cpu')) # 'cuda'
print(torch.ones(3).cpu())
print(torch.ones(3).to('cpu'))

print(torch.ones(3).to('cpu').type())

tensor([1., 1., 1.])
tensor([1., 1., 1.])
tensor([1., 1., 1.])
torch.FloatTensor


## Operations on tensors

In [12]:
a = torch.randn(3, 1)
b = torch.randn_like(a)

print("a + b = {}".format(a + b))

a + b = tensor([[ 0.4055],
        [ 0.1955],
        [-0.4838]])


In [13]:
# about ampersand operator: https://www.python.org/dev/peps/pep-0465/
print("a @ b.T = {}".format(a @ b.T))

a @ b.T = tensor([[ 0.0317, -0.0862, -0.2568],
        [ 0.0512, -0.1391, -0.4142],
        [ 0.0395, -0.1074, -0.3198]])


In [14]:
# dimensions inversion
print("a.T = {}".format(a.T))

a.T = tensor([[0.2997, 0.4833, 0.3732]])


In [15]:
# matrix transpose (https://pytorch.org/docs/stable/tensors.html#torch.Tensor.t)
print("a.t() = {}".format(a.t()))

a.t() = tensor([[0.2997, 0.4833, 0.3732]])


In [16]:
c = torch.randint(0, 10, size=(2, 3, 4, 5, 6))
print(c.size())
print(c.T.size())
print(c.transpose(0, 2).size())
# print(c.t().size()) # error

torch.Size([2, 3, 4, 5, 6])
torch.Size([6, 5, 4, 3, 2])
torch.Size([4, 3, 2, 5, 6])


In [17]:
print(c.dim()) # the order of the tensor
print(c.numel())
print(torch.numel(c))

5
720
720


In [18]:
d = torch.tensor([[0, 1], [2, 467]])
print(d)

tensor([[  0,   1],
        [  2, 467]])


In [19]:
print(d.view(-1, 4))

tensor([[  0,   1,   2, 467]])


In [20]:
print(d.dim())

2


In [21]:
print(d.unsqueeze(0).size())

torch.Size([1, 2, 2])


In [22]:
print(d.squeeze(0))
print(d.fill_(33))

tensor([[  0,   1],
        [  2, 467]])
tensor([[33, 33],
        [33, 33]])


In [23]:
d

tensor([[33, 33],
        [33, 33]])

In [24]:
d.resize_(4)

tensor([33, 33, 33, 33])

In [25]:
e = torch.ones_like(d)
print(torch.cat([d, e], dim=0))

tensor([33, 33, 33, 33,  1,  1,  1,  1])


In [26]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device: {}'.format(device))

Device: cpu


The [.to()](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.to) method returns a new tensor. We can use it to convert either its type and its device

In [27]:
e = e.to(device)

### Resources

Pictures from: [Deep Learning with Pytorch - Stevens, Antiga](https://pytorch.org/assets/deep-learning/Deep-Learning-with-PyTorch.pdf)

[Official documentation](https://pytorch.org/docs/stable/index.html)

[Tutorials](https://pytorch.org/tutorials/)

[Nice guy talking about tensors](https://www.youtube.com/watch?v=f5liqUk0ZTw)